In [1]:
#####
##IMPORT NECESSARY LIBRARIES
#####

import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from __future__ import print_function
import sys
from operator import add
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession.builder.appName('mrao999-test').getOrCreate()
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType
)

In [2]:

##### READ THE CSV FILE
##### DATAFRAME: kickstart
kickstart= spark.read.option("multiline","true").option("header", "true").option("delimiter", ",").option("inferSchema", "true").csv('./MasterKickstarter.csv')

In [3]:
##### OBTAIN THE ROW/RECORD COUNT
kickstart.count()


102561

In [4]:
##### OBTAIN THE COLUMN COUNT
len(kickstart.columns)

57

In [5]:
##### PRINT THE SCHEMA DETAILS OF THE CSV DATA
kickstart.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- X1: string (nullable = true)
 |-- X1_1: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- blurb: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- pledged: string (nullable = true)
 |-- status: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- disable_communication: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- currency_symbol: string (nullable = true)
 |-- currency_trailing_code: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- state_changed_at: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- launched_at: string (nullable = true)
 |-- staff_pick: string (nullable = true)
 |-- backers_count: string (nullable = true)
 |-- deadlineTime: string (nullable = true)
 |-- state_changed_atTime: string (nullable = true)
 |-- crea

In [6]:
##### CREATE SQL VIEW 
kickstart.createOrReplaceTempView('kickstartsqltab')


In [7]:
##### NUMBER AND STATUS OF PROJECTS IN A 'LIVE' STATUS
results = spark.sql("SELECT status,categories,count(*) as projectCount FROM kickstartsqltab where status='live' group by status,categories ")
results.show()

+------+----------------+------------+
|status|      categories|projectCount|
+------+----------------+------------+
|  live|      technology|        1112|
|  live|      journalism|         111|
|  live|         fashion|         855|
|  live|          design|        2149|
|  live|      publishing|        1269|
|  live|           music|         862|
|  live|          comics|         222|
|  live|           dance|          47|
|  live|film%20&%20video|        1257|
|  live|           games|        1796|
|  live|         theater|         109|
|  live|            food|         220|
|  live|             art|         200|
|  live|          crafts|          92|
|  live|     photography|         128|
+------+----------------+------------+



In [8]:
##### NUMBER AND STATUS OF PROJECTS IN A 'SUCCESSFULL' STATUS
results = spark.sql("SELECT status,categories,count(*) as projectCount FROM kickstartsqltab where status='successful' group by status,categories ")
results.show()

+----------+----------------+------------+
|    status|      categories|projectCount|
+----------+----------------+------------+
|successful|          comics|         694|
|successful|             art|        3568|
|successful|          crafts|         137|
|successful|film%20&%20video|        3210|
|successful|           music|       11019|
|successful|            food|         789|
|successful|           games|        1951|
|successful|          design|        1499|
|successful|      publishing|        6544|
|successful|     photography|         681|
|successful|           dance|         133|
|successful|      journalism|         110|
|successful|         fashion|         813|
|successful|      technology|        2805|
|successful|         theater|        1069|
+----------+----------------+------------+



In [9]:
##### NUMBER AND STATUS OF PROJECTS IN A 'CANCELED' STATUS
results = spark.sql("SELECT status,categories,count(*) as projectCount FROM kickstartsqltab where status='canceled' group by status,categories ")
results.show()

+--------+----------------+------------+
|  status|      categories|projectCount|
+--------+----------------+------------+
|canceled|           music|         946|
|canceled|             art|         641|
|canceled|      journalism|         101|
|canceled|           games|         193|
|canceled|         theater|         129|
|canceled|      publishing|         465|
|canceled|          crafts|          82|
|canceled|          comics|          46|
|canceled|     photography|         220|
|canceled|         fashion|         123|
|canceled|          design|          37|
|canceled|      technology|        1334|
|canceled|film%20&%20video|         664|
|canceled|           dance|           8|
|canceled|            food|         242|
+--------+----------------+------------+



In [10]:
##### NUMBER AND STATUS OF PROJECTS IN A 'SUSPENDED' STATUS
results = spark.sql("SELECT status,categories,count(*) as projectCount FROM kickstartsqltab where status='suspended' group by status,categories ")
results.show()

+---------+----------------+------------+
|   status|      categories|projectCount|
+---------+----------------+------------+
|suspended|             art|          24|
|suspended|           music|          41|
|suspended|      technology|         113|
|suspended|      journalism|           8|
|suspended|     photography|          13|
|suspended|         theater|           3|
|suspended|film%20&%20video|          18|
|suspended|         fashion|           5|
|suspended|           games|          19|
|suspended|          design|           1|
|suspended|          crafts|           8|
|suspended|      publishing|          16|
|suspended|            food|          27|
+---------+----------------+------------+



In [11]:
##### NUMBER AND STATUS OF PROJECTS IN A 'FAILED' STATUS
results = spark.sql("SELECT status,categories,count(*) as projectCount FROM kickstartsqltab where status='failed' group by status,categories ")
results.show()

+------+----------------+------------+
|status|      categories|projectCount|
+------+----------------+------------+
|failed|     photography|        1366|
|failed|           music|        8348|
|failed|      journalism|         506|
|failed|      technology|        7430|
|failed|          crafts|         399|
|failed|      publishing|       10821|
|failed|           games|        1235|
|failed|          design|         199|
|failed|film%20&%20video|        3126|
|failed|         theater|         723|
|failed|             art|        4984|
|failed|         fashion|        1103|
|failed|          comics|         315|
|failed|           dance|          63|
|failed|            food|        2166|
+------+----------------+------------+



In [12]:

##### NUMBER OF COLUMNS/RECORDS THAT HAS EITHER 'NA' OR 'NULL' STATUS

from pyspark.sql.functions import isnan,when, count,col
kickstart.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in kickstart.columns]).show()

+---+---+----+-------+----+---+----+-----+----+-------+------+----+---------------------+--------+---------------+----------------------+--------+----------------+----------+-----------+----------+-------------+------------+--------------------+--------------+---------------+----------+---------+----------+------+----------+------------------+------------+-------------+---------+-----------------+-----------+------------+-----------------+---------+--------------+--------+--------+---------+------+-----+----------------------+-------------------------+------------------------+-------------------------+--------------------------+-----------------+------------------+---------------------+----------------+------------------+-------------------+
|_c0| X1|X1_1|Country|City| id|name|blurb|goal|pledged|status|slug|disable_communication|currency|currency_symbol|currency_trailing_code|deadline|state_changed_at|created_at|launched_at|staff_pick|backers_count|deadlineTime|state_changed_atTime|cre

In [13]:
##### LIST OF COLUMNS THAT CAN BE DROPPED FROM FURTHER ANALYSIS
to_drop= [
          '_c0','X1','X1_1','id','slug','currency_symbol','currency_trailing_code','disable_communication','staff_pick',
            'state_changed_atTime','created_atTime','launched_atTime','spotLight','Ex_USd','Latitude','Longitude',
        'deadlineYM','state_changed_atYM','created_atYM','launched_atYM','Backers_as_Prct_of_Pop_YM','Backers_in_city_YM',
        'Mean_Pledge_City','Mean_pledge_city_Y','Mean_pledge_city_YM'
         ]



In [14]:

##### DROP THE UNNECESSARY COLUMNS
##### DATAFRAME: KSprep

KSprep=kickstart.drop(*to_drop)

In [15]:
##### NUMBER OF COLUMNS BEFORE DROPPING COLUMNS
##### DATAFRAME: kickstart
len(kickstart.columns)

57

In [16]:
##### NUMBER OF COLUMNS BEFORE AFTER COLUMNS
##### DATAFRAME: KSprep
len(kickstart.columns)
len(KSprep.columns)

32

In [17]:
##### NUMBER OF RECORDS BEFORE DROPPING 'NA'
##### DATAFRAME: kickstart
len(kickstart.columns)
kickstart.count()

102561

In [18]:
##### NUMBER OF RECORDS AFTER DROPPING 'NA'
##### DATAFRAME: KSprep
KSprep.na.drop().count()

96287

In [19]:
##### ADD COLUMN FOR VISUALISATION AND PATTERN ANALYSIS
##### DATAFRAME: KSprepVar
from pyspark.sql.functions import length
KSprepVar=KSprep.withColumn('BlurbLen',length(KSprep.blurb))

In [20]:
##### VIEW THE NEW DATA
KSprepVar.select('blurb','BlurbLen').show(10)

+--------------------+--------+
|               blurb|BlurbLen|
+--------------------+--------+
|A 100 page Artboo...|     134|
|Wordsmith! The st...|     133|
|The Postnatalist ...|     135|
|Alkandros Designs...|     127|
|A feature length ...|     117|
|Homemade Rustic P...|     113|
|A superhero comic...|     132|
|Cheerin is an exc...|     133|
|"We aim to show p...|     120|
|Profoundly deaf c...|     131|
+--------------------+--------+
only showing top 10 rows



In [21]:
##### READ DATA FROM NEW AND CLEAN CSV
##### DATAFRAME: kickstart_final
kickstart_final= spark.read.option("multiline","true").option("header", "true").option("delimiter", ",").option("inferSchema", "true").csv('./MasterKickstarter01.csv')

In [23]:
##### READ THE SCHEMA OF NEW INPUT FILE
kickstart_final.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- X1: integer (nullable = true)
 |-- X1_1: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- goal: string (nullable = true)
 |-- pledged: double (nullable = true)
 |-- status: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- disable_communication: boolean (nullable = true)
 |-- currency: string (nullable = true)
 |-- currency_symbol: string (nullable = true)
 |-- currency_trailing_code: boolean (nullable = true)
 |-- deadline: string (nullable = true)
 |-- state_changed_at: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- launched_at: string (nullable = true)
 |-- staff_pick: boolean (nullable = true)
 |-- backers_count: integer (nullable = true)
 |-- deadlineTime: integer (nullable = true)
 |-- state_changed_atTime: integer (nullable = true)
 |-- created_atTime: integer (nullable = true)
 |-- launched_atTime: int

In [22]:
##### RE-ITERATE
#### DROP THE UNNECESSARY COLUMNS
##### DATAFRAME: kickstart_final_clean

kickstart_final_clean=kickstart_final.drop(*to_drop)
len(kickstart_final_clean.columns)

30

In [24]:
#### data preparation
#### CREATE A UDF TO CONVERT STRING TYPE 'STATUS' VALUE TO INTEGER TYPE 'STATUS_FLAG COLUMN' 

from pyspark.sql.functions import udf
from pyspark.sql.types import *

def calculate_status(status):
    if status == 'live':
     return 1
    if status=='successful':
     return 1
    if status=='failed':
     return 0
    if status=='canceled':
     return 0
    if status=='suspended':
     return 0

In [27]:
#### ADD NEW COLUMN WITH UDF
#### DATAFRAME: KSfinal
KSudf= udf(calculate_status, IntegerType())
KSfinal= kickstart_final_clean.withColumn('status_flag',KSudf(kickstart_final_clean.status))

#number of columns after adding status_flag
len(KSfinal.columns)

31

In [28]:

#### VIEW DATA OF NEW COLUMN
KSfinal.select('status','status_flag').show(10)

+----------+-----------+
|    status|status_flag|
+----------+-----------+
|    failed|          0|
|successful|          1|
|    failed|          0|
|      live|          1|
|    failed|          0|
|      live|          1|
|      live|          1|
|    failed|          0|
|successful|          1|
|successful|          1|
+----------+-----------+
only showing top 10 rows



In [29]:
#### PRINT SCHEMA DETAILS AFTER READING NEW COLUMN
KSfinal.printSchema()

root
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- pledged: double (nullable = true)
 |-- status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- state_changed_at: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- launched_at: string (nullable = true)
 |-- backers_count: integer (nullable = true)
 |-- deadlineTime: integer (nullable = true)
 |-- Categories: string (nullable = true)
 |-- pledgedUSD: double (nullable = true)
 |-- deadlineY: integer (nullable = true)
 |-- state_changed_atY: integer (nullable = true)
 |-- created_atY: integer (nullable = true)
 |-- launched_atY: integer (nullable = true)
 |-- Pledge_per_person: string (nullable = true)
 |-- Prct_goal: double (nullable = true)
 |-- Length_of_kick: integer (nullable = true)
 |-- City_Pop: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nu

In [30]:
#DATA MINING LIBRARIES

from pyspark.ml import Pipeline
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [31]:
#### PREPARE 'LABELS' AND 'FEATURES' FOR ANALYSIS

#SELECT FEATURES AND TRANSFORM TO VECTOR TYPE
assembler = VectorAssembler(
    inputCols=["pledged", "deadlineTime", "backers_count", "pledgedUSD","deadlineY","state_changed_atY","created_atY",
               "launched_atY","Prct_goal","Length_of_kick","Backers_in_city_Y","All_Time_Backers_city","Days_inception_to_Deadline","Days_spent_making_campign","City_Pop",
               "Backers_as_Prct_of_Pop","Backers_as_Prct_of_Pop_Y"
    ],
    outputCol="features")
output = assembler.transform(KSfinal)

#SELECT LABEL/TARGET ATTRIBUTE IN STRING TYPE FORMAT
labelIndexer = StringIndexer(inputCol="status", outputCol="indexedLabel").fit(output)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)

In [32]:
#### SELECT THE TEST AND TRAINING DATA SPLIT
(trainingData, testData) = output.randomSplit([0.7, 0.3])

In [33]:
#### ALGORITHM-1
#### RANDOM FOREST

#ASSIGN LABELS, FEATURES AND TREE LENGTH
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)


In [34]:
#### CONVERT BACK THE INDEX LABEL TO STRING FOR FURTHER REFERENCE
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

In [35]:
#### CREATE PIPELINE OF LABEL,FEATURE AND RF TREE
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])


In [36]:
#### RUN MODEL FOR TRAINING DATA
model = pipeline.fit(trainingData)

In [37]:
#### RUN MODEL FOR TESTING DATA AND MAKE PREDICTIONS
predictions = model.transform(testData)

#### SELECT A SAMPLE OF PREDICTIONS
predictions.select("predictedLabel","features").show(5)

+--------------+--------------------+
|predictedLabel|            features|
+--------------+--------------------+
|    successful|[1019.0,6584.0,27...|
|          live|[0.0,64023.0,0.0,...|
|        failed|[5692.0,65062.0,1...|
|    successful|[5265.0,66999.0,5...|
|          live|[243.0,28242.0,7....|
+--------------+--------------------+
only showing top 5 rows



In [38]:
#### EVALUATE THE PREDICTION AND CALCULATE ERROR IN ACCURACY
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

#MODEL DESCRIPTION
rfModel = model.stages[2]
print(rfModel) 

Test Error = 0.14235
RandomForestClassificationModel (uid=rfc_f5d6e3218e1b) with 10 trees


In [39]:
#### ALGORITHM-2
#### DECESION TREE

#ASSIGN LABELS, FEATURES AND TREE LENGTH
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [40]:
#### CREATE PIPELINE OF LABEL,FEATURE AND DT
pipeline2 = Pipeline(stages=[labelIndexer, featureIndexer, dt])


In [41]:
#### RUN MODEL FOR TRAINING DATA
model2 = pipeline2.fit(trainingData)

In [42]:
#### RUN MODEL FOR TESTING DATA AND MAKE PREDICTIONS
predictions2 = model2.transform(testData)

#### SELECT A SAMPLE OF PREDICTIONS
predictions2.select("prediction", "indexedLabel", "features").show(5)


+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         2.0|[1019.0,6584.0,27...|
|       2.0|         0.0|[0.0,64023.0,0.0,...|
|       0.0|         0.0|[5692.0,65062.0,1...|
|       1.0|         1.0|[5265.0,66999.0,5...|
|       2.0|         0.0|[243.0,28242.0,7....|
+----------+------------+--------------------+
only showing top 5 rows



In [43]:
#### EVALUATE THE PREDICTION AND CALCULATE ERROR IN ACCURACY
evaluator2 = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy2 = evaluator2.evaluate(predictions2)
print("Test Error = %g " % (1.0 - accuracy2))

#MODEL DESCRIPTION
treeModel = model2.stages[2]
print(treeModel)

Test Error = 0.143364 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_44b8abbc40dec4f78d0d) of depth 5 with 63 nodes


In [44]:
#### ALGORITHM-3
#### GRADIENT-BOOSTED TREES -GBT

#ASSIGN LABELS, FEATURES AND TREE LENGTH
labelIndexer = StringIndexer(inputCol="status_flag", outputCol="indexedLabel").fit(output)
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

In [45]:
#### CREATE PIPELINE OF LABEL,FEATURE AND GBT
pipeline3 = Pipeline(stages=[labelIndexer, featureIndexer, gbt])


In [46]:
#### RUN MODEL FOR TRAINING DATA
model3 = pipeline3.fit(trainingData)

In [47]:
#### RUN MODEL FOR TESTING DATA AND MAKE PREDICTIONS
predictions3 = model3.transform(testData)

#### SELECT A SAMPLE OF PREDICTIONS
predictions3.select("prediction", "indexedLabel", "features").show(5)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|[1019.0,6584.0,27...|
|       1.0|         0.0|[0.0,64023.0,0.0,...|
|       0.0|         0.0|[5692.0,65062.0,1...|
|       1.0|         1.0|[5265.0,66999.0,5...|
|       1.0|         0.0|[243.0,28242.0,7....|
+----------+------------+--------------------+
only showing top 5 rows



In [48]:
#### EVALUATE THE PREDICTION AND CALCULATE ERROR IN ACCURACY
evaluator3 = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy3 = evaluator3.evaluate(predictions3)
print("Test Error = %g" % (1.0 - accuracy3))

#MODEL DESCRIPTION
gbtModel = model3.stages[2]
print(gbtModel) 

Test Error = 0.0684362
GBTClassificationModel (uid=GBTClassifier_4586b7adbb069d7c6c96) with 10 trees
